In [ ]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds
from tools import EarlyStopping

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# In[2]:
torch.set_num_threads(8)
torch.get_num_threads()

In [ ]:
class SemiDataset(Dataset):
    def __init__(self, df, fea_cols, y_cols):        
        self.X = df[fea_cols].values
        self.y = df[y_cols].values
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
    

df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)

print(df_train.shape, df_test.shape)

layer_cols = [c for c in df_train.columns if 'layer_' in c]
fea_cols = [c for c in df_train.columns if c not in layer_cols]

len(fea_cols), len(layer_cols)

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(df_train[fea_cols].values)

scaler = joblib.load('scaler.bin')

df_train[fea_cols] = scaler.transform(df_train[fea_cols].values)
df_test[fea_cols] = scaler.transform(df_test[fea_cols].values)

torch.manual_seed(81511991154)
torch.initial_seed()

df_model = df_train

dataset = SemiDataset(df_model[fea_cols + layer_cols], fea_cols, layer_cols)

train_set, val_set = torch.utils.data.random_split(dataset, [700000, 110000])

print(len(train_set), len(val_set))

batch_size = 30000
num_workers = 4

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=num_workers)
val_loader = DataLoader(dataset=val_set, batch_size=10000)

print(f'batch_size {batch_size} num_workers {num_workers}')
print(f'train_loader {len(train_loader)} val_loader {len(val_loader)}')

#### Model

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
#         relu = torch.nn.ReLU()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 256), torch.nn.BatchNorm1d(256), act, dropout,            
            torch.nn.Linear(256, 4)
        )
    def forward(self, x):
        return self.model(x)
 

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 256), torch.nn.BatchNorm1d(256), act, dropout,            
            torch.nn.Linear(256, 256), torch.nn.BatchNorm1d(256), act, dropout,            
            torch.nn.Linear(256, 128), torch.nn.BatchNorm1d(128), act, dropout,            
            torch.nn.Linear(128, 4)
        )
    def forward(self, x):
        return self.model(x)
 

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
#         relu = torch.nn.ReLU()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 3000), torch.nn.BatchNorm1d(3000), act, dropout, 
             torch.nn.Linear(3000, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 1500), torch.nn.BatchNorm1d(1500), act, dropout,
            torch.nn.Linear(1500, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 256), torch.nn.BatchNorm1d(256), act, dropout,            
            torch.nn.Linear(256, 4)
        )
    def forward(self, x):
        return self.model(x)
 

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 3000), torch.nn.BatchNorm1d(3000), act, dropout, 
             torch.nn.Linear(3000, 3000), torch.nn.BatchNorm1d(3000), act, dropout, 
             torch.nn.Linear(3000, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 1500), torch.nn.BatchNorm1d(1500), act, dropout,
            torch.nn.Linear(1500, 1500), torch.nn.BatchNorm1d(1500), act, dropout,
            torch.nn.Linear(1500, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 256), torch.nn.BatchNorm1d(256), act, dropout,            
            torch.nn.Linear(256, 4)
        )
    def forward(self, x):
        return self.model(x)
 

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
#         relu = torch.nn.ReLU()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 1024), torch.nn.BatchNorm1d(1024), act, dropout, 
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 4)
        )
    def forward(self, x):
        return self.model(x)
 

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 4096), torch.nn.BatchNorm1d(4096), act, dropout, 
            torch.nn.Linear(4096, 4096), torch.nn.BatchNorm1d(4096), act, dropout,            
            torch.nn.Linear(4096, 2048), torch.nn.BatchNorm1d(2048), act, dropout,            
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout,            
            torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 4)
        )
    def forward(self, x):
        return self.model(x)
 

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
#         relu = torch.nn.ReLU()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,            
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 256), torch.nn.BatchNorm1d(256), act, dropout,                        
            torch.nn.Linear(256, 4)
        )
    def forward(self, x):
        return self.model(x)
 

In [ ]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3):
        super(DNNModel,self).__init__()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 512), torch.nn.BatchNorm1d(512), act, dropout, 
            torch.nn.Linear(512, 1024), torch.nn.BatchNorm1d(1024), act, dropout, 
            torch.nn.Linear(1024, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 2048), torch.nn.BatchNorm1d(2048), act, dropout, 
            torch.nn.Linear(2048, 1024), torch.nn.BatchNorm1d(1024), act, dropout,
            torch.nn.Linear(1024, 512), torch.nn.BatchNorm1d(512), act, dropout,
            torch.nn.Linear(512, 4)
        )
    def forward(self, x):
        return self.model(x)
 

In [ ]:
model = DNNModel(input_size=226, dropout_probability=0).to(device)

# model.load_state_dict(torch.load('model/20200123T082145_0.5488672057787577.model'))

model.load_state_dict(torch.load('model/checkpoint.pt'))
# model.load_state_dict(torch.load('model/20200125T200128_0.4469637870788574.pt'))


model.to(device)
model

#### Pred Valid

In [ ]:
criterion = nn.L1Loss(reduction='mean').to(device)
model.eval()
total_loss = 0
for data in val_loader:
    X_batch, y_batch = data
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)
    
    with torch.no_grad():
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        total_loss = total_loss + loss.item()
        
val_loss_min = total_loss / len(val_loader)
val_loss_min

#### Pred Test 

In [ ]:
model.eval()
y_pred = model(torch.Tensor(df_test[fea_cols].values).to(device))    
print(y_pred)

ts = datetime.now().strftime('%Y%m%dT%H%M%S')

df_submit = pd.read_csv('input/sample_submission.csv', index_col=0)

df_submit[layer_cols] = y_pred.cpu().detach().numpy()
df_submit.to_csv(f'submit/{ts}_{val_loss_min}.csv')

print(ts, val_loss_min)

In [ ]:
# model.load_state_dict(torch.load('model/20200123T215856_0.6811393996079763.model'))